Предсказание решения математического маятника.

In [ ]:
import tensorflow as tf
import numpy as np
import time
import matplotlib
from matplotlib import pyplot
%run lstm.ipynb

## Гиперпараметры

In [ ]:
dec = 1000

num_epochs = 30
num_repeat = 1

optimizer = 'adagrad'
learn_rate = 0.5

input_dim = 30
hidden_dim = 30
output_dim = input_dim

do_svd = True
svd_iters = 16
svd_begin = 0.
svd_stren = svd_begin
svd_weaken = 0.07

## Чтение данных

In [ ]:
path = 'e:/pendulum.txt'
data = np.genfromtxt(path)

#x = data[:,0][::dec]
y = data[:,1][::dec]

matplotlib.pyplot.plot(y)

## Подготовка данных

In [ ]:
def split(data, ratio):
    l = int(len(data) * ratio)
    return data[:l], data[l:]

In [ ]:
def shift(data, dim, step):
    series = []
    for i in range(0, len(data) - dim, step):
        series += [data[i:i+dim]]
    return series

In [ ]:
y_train, y_test = split(y, 0.8)

y_train_in = shift(y_train[0:], input_dim, 1)
y_train_out = shift(y_train[1:], input_dim, 1)

#matplotlib.pyplot.plot(y_train)
#matplotlib.pyplot.plot(y_test)

## Обучение

In [ ]:
def train(lstm, data_in, data_out, num_epochs, num_repeat):
    time_begin = time.time()
    for e in range(num_epochs):
        lstm.clear_state()
        for rec_in, rec_out in zip(data_in, data_out):
            lstm.train([rec_in], [rec_out], num_repeat)
    return time.time() - time_begin

In [ ]:
def test(lstm, begin, num_steps, take=-1):
    points = []
    prev = [begin]
    for i in range(num_steps):
        prev = lstm.test(prev)
        points += [prev[-1][take]]
    return points

In [ ]:
sess = tf.Session()

In [ ]:
lstm = lstm_cell(sess, input_dim, hidden_dim, output_dim, learn_rate, optimizer)

In [ ]:
train_time = []
svd_norm = []

In [ ]:
for s in range(svd_iters):
    train_time += [train(lstm, y_train_in, y_train_out, num_epochs, num_repeat)]
    lstm.save_all()
    if do_svd:
        norm, size = lstm.svd_compress(svd_stren, svd_stren)
        svd_norm += [norm]
        svd_stren += svd_weaken
    print(s, end=' ')
lstm.restore_all()

## Предсказание

In [ ]:
lstm.restore_all()
pred = test(lstm, y_train_in[-1], 100)

In [ ]:
matplotlib.pyplot.plot(pred) # <= син
matplotlib.pyplot.plot(y_test) # <= оранж